In [14]:
%%writefile merg_sort.cpp
#include <iostream>
#include <omp.h>
using namespace std;
#define ARRAY_SIZE 256
#define NUM_THREADS 8

void swap(int* a, int* b) {
    int temp = *a;
    *a = *b;
    *b = temp;
}

int partition(int* arr ,int low, int high) {
    int pivot = arr[high];
    int i = (low - 1);

    for (int j = low; j <= high - 1; j++) {
        if (arr[j] < pivot) {
            i++;
            swap(&arr[i], &arr[j]);
        }
    }
    swap(&arr[i + 1], &arr[high]);
    return (i + 1);
}

void quickSort(int* arr, int low, int high) {
    if (low < high) {
        int pi = partition(arr, low, high);
        quickSort(arr, low, pi - 1);
        quickSort(arr, pi + 1, high);
    }
}

void merge(int* arr, int low, int mid, int high) {
    int* temp = new int[high-low + 1];
    int i = low, j = mid + 1, k = 0;

    while (i <= mid && j <= high) {
        if (arr[i] < arr[j])
            temp[k++] = arr[i++];
        else
            temp[k++] = arr[j++];
    }

    while (i <= mid)
        temp[k++] = arr[i++];

    while (j <= high)
        temp[k++] = arr[j++];

    for (i = low; i <= high; i++)
        arr[i] = temp[i - low];
    delete[] temp;
}

int main() {
    int* arr = new int[ARRAY_SIZE];
    printf("Unsorted array: \n");
    for (int i = 0; i < ARRAY_SIZE; i++) {
        arr[i] = rand() % 100;  // Random values between 0 and 99
        printf("%d ", arr[i]);
    }
    int chunk_size = ARRAY_SIZE / NUM_THREADS;
    int i;


    #pragma omp parallel num_threads(NUM_THREADS)
    {
        printf("thread index in quick sort is : %d \n",omp_get_thread_num());
        int thread_num = omp_get_thread_num();
        int start_index = thread_num * chunk_size;
        int end_index = start_index + chunk_size - 1;

        quickSort(arr, start_index, end_index);
    }

    // Merge the sorted parts in parallel
    int step = 1;
    int thread_step = 2;
    while (step < NUM_THREADS) {
        #pragma omp parallel for num_threads(NUM_THREADS/thread_step)
        for (i = 0; i < NUM_THREADS; i += 2 * step) {
            printf("thread index is : %d \n",omp_get_thread_num());
            int start_index = i * chunk_size;
            int mid_index = (i + step) * chunk_size - 1;
            int end_index = (i + 2 * step) * chunk_size - 1;


            merge(arr, start_index, mid_index, end_index);
        }

        step *= 2;
        thread_step *= 2;
    }

    // Print the sorted array
    printf("Sorted array: \n");
    for (i = 0; i < ARRAY_SIZE; i++)
        printf("%d ",arr[i]);
    printf("\n");

    delete[] arr;
    return 0;
}


Overwriting merg_sort.cpp


In [15]:
! g++ merg_sort.cpp -o execfile -fopenmp

In [16]:
! ./execfile

Unsorted array: 
83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 36 5 46 29 13 57 24 95 82 45 14 67 34 64 43 50 87 8 76 78 88 84 3 51 54 99 32 60 76 68 39 12 26 86 94 39 95 70 34 78 67 1 97 2 17 92 52 56 1 80 86 41 65 89 44 19 40 29 31 17 97 71 81 75 9 27 67 56 97 53 86 65 6 83 19 24 28 71 32 29 3 19 70 68 8 15 40 49 96 23 18 45 46 51 21 55 79 88 64 28 41 50 93 0 34 64 24 14 87 56 43 91 27 65 59 36 32 51 37 28 75 7 74 21 58 95 29 37 35 93 18 28 43 11 28 29 76 4 43 63 13 38 6 40 4 18 28 88 69 17 17 96 24 43 70 83 90 99 72 25 44 90 5 39 54 86 69 82 42 64 97 7 55 4 48 11 22 28 99 43 46 68 40 22 11 10 5 1 61 30 78 5 thread index in quick sort is : 5 
thread index in quick sort is : 3 
thread index in quick sort is : 4 
thread index in quick sort is : 6 
thread index in quick sort is : 7 
thread index in quick sort is : 0 
thread index in quick sort i